In [1]:
import numpy as np
from time import time
from loguru import logger
import sys

In [2]:
raw_data = open('files/input_day8.txt').read()

In [3]:
example = '''162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689'''

In [4]:
def preprocess(data):
    return [[int(n) for n in jb.split(',')] for jb in data.strip().split('\n')]
preprocess(example)

[[162, 817, 812],
 [57, 618, 57],
 [906, 360, 560],
 [592, 479, 940],
 [352, 342, 300],
 [466, 668, 158],
 [542, 29, 236],
 [431, 825, 988],
 [739, 650, 466],
 [52, 470, 668],
 [216, 146, 977],
 [819, 987, 18],
 [117, 168, 530],
 [805, 96, 715],
 [346, 949, 466],
 [970, 615, 88],
 [941, 993, 340],
 [862, 61, 35],
 [984, 92, 344],
 [425, 690, 689]]

In [ ]:
def calculate_single_distance(pos_A, pos_B):
    xA, yA, zA = pos_A
    xB, yB, zB = pos_B
    distance = float(np.sqrt((xA-xB)**2 + (yA-yB)**2 + (zA-zB)**2))
    return distance
    
def get_all_distances(positions):
    distances = {}
    for a, pos_A in enumerate(positions):
        for b, pos_B in enumerate(positions):
            if a == b or distances.get((b,a)) is not None:
                continue
            distances[(a, b)] = calculate_single_distance(pos_A, pos_B)
    return distances

def get_top_min_distances(dist_dict, top = 10):
    dist_list = [v for v in dist_dict.values()]
    dist_list.sort()
    sorted_dist_dict = {}
    for sv in dist_list[:top]:
        for k, v in dist_dict.items():
            if sv == v and sorted_dist_dict.get(k) is None:
                sorted_dist_dict[k] = v
                break
    return sorted_dist_dict

In [32]:
positions = preprocess(example)
dist_dict = get_all_distances(positions)
min_distances = get_top_min_distances(dist_dict, top = 10)
min_distances

{(0, 19): 316.90219311326956,
 (0, 7): 321.560258738545,
 (2, 13): 322.36935338211043,
 (7, 19): 328.11888089532425,
 (17, 18): 333.6555109690233,
 (9, 12): 338.33858780813046,
 (11, 16): 344.3893145845266,
 (2, 8): 347.59890678769403,
 (14, 19): 350.786259708102,
 (2, 18): 352.936254867646}

In [ ]:
def find_box_in_circuits(box_id, circuits):
    box_circ_indx = None
    for i, c in enumerate(circuits):
        if box_id in c:
            box_circ_indx = i
            break
    return box_circ_indx

# def find_unconnected_boxes(circuits, total_boxes):
#     connected = list(set(b for circ in circuits for b in circ))
#     unconnected [b_id for b_id in range(total_boxes) if b_id not in connected]
#     return unconnected

def connect_boxes(min_distances, total_boxes):
    circuits = []
    for (a, b) in min_distances.keys():
        a_circ_indx = find_box_in_circuits(a, circuits)
        b_circ_indx = find_box_in_circuits(b, circuits)
        if a_circ_indx is None and b_circ_indx is None:
            circuits.append([a, b])
        elif a_circ_indx == b_circ_indx:
            continue
        elif a_circ_indx is None:
            logger.debug(f'Joining {a} to {b} circuit: {circuits[b_circ_indx]}')
            circuits[b_circ_indx].append(a)
        elif b_circ_indx is None:
            circuits[a_circ_indx].append(b)
        else:
            logger.debug(f'Joining 2 circuits: {circuits[a_circ_indx]} and {circuits[b_circ_indx]} because of {(a,b)}')
            circuits[a_circ_indx].extend(circuits[b_circ_indx])
            circuits.pop(b_circ_indx)
    return circuits

def get_result(circuits):
    circ_len = [len(c) for c in circuits]
    circ_len.sort(reverse = True)
    result = circ_len[0] * circ_len[1] * circ_len[2]
    return result

def solve(data, top = 10):
    t_start = time()

    positions = preprocess(data)
    dist_dict = get_all_distances(positions)
    min_distances = get_top_min_distances(dist_dict, top = top)
    circuits = connect_boxes(min_distances, len(positions))
    logger.debug(circuits)
    result = get_result(circuits)

    t_end = time()
    logger.info(f'Part 1 took: {(t_end-t_start)*1000:.2f}ms')
    logger.info(f'Result is = {result}')



In [ ]:
solve(example, top = 10)

2025-12-08 22:09:17.512 | DEBUG    | __main__:connect_boxes:28 - Joining 2 circuits: [2, 13, 8] and [17, 18] because of (2, 18)
2025-12-08 22:09:17.513 | DEBUG    | __main__:solve:46 - [[0, 19, 7, 14], [2, 13, 8, 17, 18], [9, 12], [11, 16]]
2025-12-08 22:09:17.513 | INFO     | __main__:solve:50 - Part 1 took: 1.55ms
2025-12-08 22:09:17.513 | INFO     | __main__:solve:51 - Result is = 40


In [57]:
logger.remove()
logger.add(sys.stderr, level="INFO")
solve(raw_data, top = 1000)

2025-12-08 22:12:01.862 | INFO     | __main__:solve:50 - Part 1 took: 3591.96ms
2025-12-08 22:12:01.862 | INFO     | __main__:solve:51 - Result is = 181584


## Part 2